# Benchmark
This notebook contains the code that is used to evaluate quantitatively and visualize the results of the analyzed methods.

## Importing libraries

In [1]:
import cv2
import os
from math import ceil
import numpy as np
import pandas as pd
import ipywidgets as widgets
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import ipynb.fs.defs.FeatureMatching as FeatureMatching
import ipynb.fs.defs.GraphBuilding as GraphBuilding
import ipynb.fs.defs.SimpleGraphStitching as SimpleGraphStitching
import ipynb.fs.defs.Utils as Utils
from IPython.display import display
import IPython

## Variables definition

In [2]:
BENCHMARK_DIR = "benchmark" #Directory where to save the results
FILENAME = "results.csv" #Name of the file that will contain the results
INDEX = ["number_of_matches", "noise_std"] #Set of attributes that is used as index in the results file

## Functions definition

In [3]:
#This function allows to estimate the error between two homographies
def compute_error(H_gt, #Ground truth homography
                  H #Estimated homography
                 ):
    #Compute the determinant of the ground truth homography and normalize it so that the determinant is unitary
    det_H_gt = np.linalg.det(H_gt)
    H_gt = H_gt/np.cbrt(det_H_gt)
    
    #Compute the determinant of the estimated homography and normalize it so that the determinant is unitary
    det_H = np.linalg.det(H)
    H = H/np.cbrt(det_H)
    
    return np.linalg.norm(np.eye(3)-H_gt@np.linalg.inv(H)) #Compute and return the error

In [4]:
#This function allows to compute the average error given a set of estimated homographies and ground truth homographies
def compute_avg_error(H_gt, #Ground truth homography
                      H #Estimated homography
                     ):
    return np.mean([compute_error(H_gt_node, H_node) for H_gt_node, H_node in zip(H_gt,H)]) #Return average error

In [5]:
#This function allows to compute a ground truth set of homographies associated to a stitched image
def compute_ground_truth(dataset_name, #Name of the dataset to be used
                         imgs, #Images to be stitched
                         T_norm, #Normalization matrix
                         matching_threshold, #Threshold used to compute matches
                         matches_th, #Threshold used to compute good matches
                         idx_ref #Index of the reference image
                        ):
    #Perform feature matching
    matches_dict_gt, _ = FeatureMatching.get_feature_matches(dataset_name,
                        imgs,
                        T_norm,
                        matching_threshold = matching_threshold,
                        number_of_matches = 1,
                        matches_th = matches_th,
                        RANSACmaxIters = 2000,
                        save_output = False,
                        save_images = False,
                        noise_type = Utils.NoiseType.NO_NOISE,
                        verbose = False
                       )

    #Produce the ground truth using simple graph stitching in a noise-free scenario
    #Compute matrices needed to perform simple graph stitching
    M_gt, _, _ = GraphBuilding.build_simple_graph_matrices(dataset_name,
                imgs,
                matches_dict_gt,
                verbose = False,
                save_output = False
               )

    #Apply simple graph stitching to get the ground truth homographies and the stitched image
    H_gt, stitched_image_gt,_ = SimpleGraphStitching.simple_graph_stitching(dataset_name,
                                imgs,
                                T_norm,
                                M_gt, 
                                idx_ref = idx_ref,
                                verbose = False,
                                save_output = False,
                                beautify = True,
                                warp_shape = [10000,10000])
    
    return H_gt, stitched_image_gt

In [6]:
#This function allows to visualize the statistics related to the results
def visualize_stats(df):
    #Sort results according to the index
    df.sort_values(by=INDEX, ascending=True, inplace=True)
    display(df)
    
    #Drop the column related to the number of experiments since it's not needed
    df_v = df.drop(columns="Experiments")
    
    noises = np.unique([ v[1] for v in df_v.index.values]) #Set of noise values
    matches = np.unique([ v[0] for v in df_v.index.values]) #Set of multi-edge degree values

    #Plot representations
    Utils.scatter_with_slider(noises, df_v.reorder_levels(['noise_std','number_of_matches']).sort_index(), "Multi-edge degree", "Noise std")
    Utils.scatter_with_slider(matches, df_v, "Noise std", "Multi-edge degree")
    Utils.mesh_plot(df_v)


In [7]:
#This function allows to visualize the qualitative and quantitative results
def visualize_results(df, results, stitched_img_gt):
    #Plot the obtained stitched images
    Utils.plot_images(results, stitched_img_gt)
    
    #Plot qualitative results
    visualize_stats(df)
    

In [8]:
#This function allows to load the results obtained for a specific experiment
def load_gt_and_stats(dataset_dir, #Directory of the dataset to be used
                df_file, #Name of the file that contains the results
                dataset_name, #Name of the dataset to be used
                imgs, #Images to be stitched
                T_norm, #Normalization matrix
                matching_threshold, #Threshold used to compute matches
                matches_th, #Threshold used to compute good matches
                idx_ref, #Index of the reference image
                results #List of dictionaries containing the results for each method
                ):
    
    index = INDEX
    #Define where to save ground truth data
    gt_file = os.path.join(dataset_dir,"gt.npy")
    gt_img_file = os.path.join(dataset_dir,"gt_img.jpg")
    
    #Load the GT if it exists or compute it
    if os.path.isfile(gt_file):
        H_gt = np.load(gt_file)
        stitched_img_gt = cv2.cvtColor(cv2.imread(gt_img_file), cv2.COLOR_BGR2RGB)
        df = pd.read_csv(df_file, index_col = index)
    else:
        H_gt, stitched_img_gt = compute_ground_truth(dataset_name,imgs,T_norm,matching_threshold,matches_th, idx_ref)
        cv2.imwrite(gt_img_file, cv2.cvtColor(stitched_img_gt, cv2.COLOR_RGB2BGR))
        np.save(gt_file,H_gt)
        columns = index + [r["name"] for r in results] + ["Experiments"]
        df = pd.DataFrame(columns=columns)
        df.set_index(index,inplace=True)
        
    return H_gt, stitched_img_gt, df

In [9]:
#This function allows to compute the statistics of a certain experiment
def run_benchmark(dataset_name, #Name of the dataset to be used
                  imgs, #Images to be stitched
                  T_norm, #Normalization matrix
                  matching_threshold,  #Threshold used to compute matches
                  matches_th, #Threshold used to compute good matches
                  idx_ref, #Index of the reference image
                  results, #List of dictionaries containing the results for each method
                  number_of_matches, #Multi-edge degree value
                  noise_std, #Standard deviation of the noise
                  noise_type, #Type of noise added during the experiments
                  visualize = True #If True allows to plot the results of the experiment
                 ):
    
    benchmark_dir = BENCHMARK_DIR
    filename = FILENAME
    
    #Compute the dataset directory path
    dataset_dir = os.path.join(benchmark_dir,f"{dataset_name}_{idx_ref}_{matching_threshold}_{matches_th}_{noise_type.name}")
  
    #Compute the path of the file where to save results
    df_file = os.path.join(dataset_dir, filename)
    os.makedirs(dataset_dir, exist_ok = True)
    
    #Load the results of the experiments
    H_gt, stitched_img_gt, df = load_gt_and_stats(dataset_dir,
                df_file,
                dataset_name,
                imgs,
                T_norm,
                matching_threshold,
                matches_th, 
                idx_ref,
                results)
    
    #Average error of the experiments with the same noise std and multi-edge degree
    index = (number_of_matches, noise_std)
    if index in df.index:
        #Update number of experiments
        exp = df.loc[index,'Experiments'] + 1
        #Update average considering the new experiment's results
        new_row = [df.loc[index,r["name"]] * ((exp-1)/exp) + compute_avg_error(H_gt,r["H"])*1/exp for r in results]
        new_row.append(exp)
    else:
        #Compute results
        new_row = [compute_avg_error(H_gt,r["H"]) for r in results]
        #Number of experiments is 1
        new_row.append(1)
    
    #Save the updated results
    df.loc[index,:] = new_row
    df.to_csv(df_file)
    
    #Visualize results if required
    if visualize:
        visualize_results(df, results, stitched_img_gt)


In [10]:
#This function allows to design a dropdown menu from which a set of experiments can be selected
def dropdown_stats_eventhandler(change):
    with output:
        selected = change.new
        file = os.path.join(BENCHMARK_DIR, os.path.join(selected, FILENAME))
        
        #Read the results of the chosen set of experiments
        df = pd.read_csv(file, index_col = INDEX)

        #Print the results 
        visualize_stats(df)
        IPython.display.clear_output(wait=True)

## Visualize Statistics
It is useful to evaluate performances of methods in a dataset without computing a new benchmark.

In [14]:
#Find existing set of experiments
directories = os.listdir(BENCHMARK_DIR)
output = widgets.Output()

#Print dropdown menu
dropdown_stats = widgets.Dropdown(options = directories, value=None, description='Benchmarks:')
dropdown_stats.observe(dropdown_stats_eventhandler, names='value') 
display(dropdown_stats, output)
IPython.display.clear_output(wait=True)

Dropdown(description='Benchmarks:', options=('Helens_0_0.7_30_POINTS', 'mountain_0_0.7_30_POINTS', 'mountain_a…

Output()